# import 

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder

# load the dataset

In [2]:
df = pd.read_csv('dataset.csv')
df = df[['Date' , 'HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']]


In [7]:
heat_map = df.drop(['HomeTeam', 'AwayTeam','Date'], axis=1)
heat_map = pd.get_dummies(heat_map)

In [8]:
fig = px.imshow(heat_map.corr())
fig.show()